In [1]:
#Importing libraries
from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate

c:\Users\dell\OneDrive\Documents\LangChain and Vector database\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
# Our database will contain some techincal articles 
# We will store the content from the following urls
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

Loading documents and spliting into chunks

In [3]:
# using seleniumurlloader
loader= SeleniumURLLoader(urls=urls)
loaded_docs = loader.load()

# using text splitter
splitter = CharacterTextSplitter(chunk_size = 1000,
                                 chunk_overlap=50)
docs = splitter.split_documents(loaded_docs)

Created a chunk of size 1226, which is longer than the specified 1000


Computing Embeddings and storing in Deeplake store

In [ ]:
embeddings=GooglePalmEmbeddings()

# Creating a Deeplake database
my_activeloop_org_id = "samman"
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embeddings)

db.add_documents(docs)

In [6]:

query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Home > Tech > How to Check Disk Usage in Linux (4 Methods)

How to Check Disk Usage in Linux (4 Methods)

Beebom Staff

Updated: December 19, 2023

Comments							
							
								0

Share

Copied

There may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.

Table of Contents

Display Disk Usage in Human Readable Format

Display Disk Occupancy of a Particular Type

Display Disk Usage in Human Readable Format

Display Disk Usage for a Particular Directory

Compare Disk Usage of Two Directories

Sorting Files based on File Size

Exclude Files Based on Their File Size

Exclude Files Based on their Types


Preparing the template

In [8]:
template = """You are a well experienced customer support chatbot that helps user with tech related questions.
You provide answer in a simple and easily understanble language. 
You have knowledge of the following context:

{chunks_formatted}

Use the information provided to answer the questions from a customer. Use only the information provided and dont invent the stuffs yourself.
If the question can not be answered with the information provided simply answer "I donot know the answer."

Question: {query}
Answer:
"""

prompt = PromptTemplate(
    input_variables=['chunks_formatted', 'query'],
    template = template,
)

The working pipeline

In [18]:
# user questions
query = "How to check disk usage in linux?"

# Retrieveling relevant chunks
relevant_docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in relevant_docs]

In [19]:
# Formating the chunks
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

In [20]:
# Generating the answer
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0)
answer = llm.invoke(prompt_formatted)
print(answer)

content='There are four methods to check disk usage in Linux:\n\n1. Check Disk Space Using the df Command\n2. Check Disk Usage Using Gnome Disk Tool\n3. Check Disk Usage Using Disk Usage Analyzer Tool\n4. Cleanup Disk using Disk Usage Analyzer'
